
<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Local Search <br>


____

# Problem definition

Suppose we have a graph with 20 nodes and an edge between each pair of nodes. We aim to pick 5 nodes from the graph that induced subgraph of these nodes has a high sum of edges. To do that we will implement 3 algorithms.First, **naive(brute force)** and Second, **Hill-Climbing**, and then  **Simulated-Annealing**.

In [1]:
import numpy as np 
import random
import math

# Generator
Generator, generate a graph with n nodes. edges matrix is an n*n matrix that edges[i][j] is the weight of the edge between nodes i and j.

In [144]:
def Generator(n, seed=42) :
    random.seed(seed)
    edges = np.zeros((n,n))
    for i in range(n) : 
        for j in range(n) :
            if i >= j : 
                continue 
            edges[i][j] = random.randint(0,100)
            edges[j][i] = edges[i][j]
    return edges

In [145]:
n = 20
edges = Generator(n)

In [146]:
print(edges)

[[ 0. 81. 14.  3. 94. 35. 31. 28. 17. 94. 13. 86. 94. 69. 11. 75. 54.  4.
   3. 11.]
 [81.  0. 27. 29. 64. 77.  3. 71. 25. 91. 83. 89. 69. 53. 28. 57. 75. 35.
   0. 97.]
 [14. 27.  0. 20. 89. 54. 43. 35. 19. 27. 97. 43. 13. 11. 48. 12. 45. 44.
  77. 33.]
 [ 3. 29. 20.  0.  5. 93. 58. 68. 15. 48. 10. 70. 37. 80. 79. 46. 73. 24.
  90.  8.]
 [94. 64. 89.  5.  0.  5. 84. 29. 98. 37. 10. 29. 12. 48. 35. 58. 81. 46.
  20. 47.]
 [35. 77. 54. 93.  5.  0. 45. 26. 85. 34. 89. 87. 82.  9. 77. 81. 21. 68.
  93. 31.]
 [31.  3. 43. 58. 84. 45.  0. 20. 59. 48. 34. 81. 88. 71. 28. 87. 41. 98.
  99.  7.]
 [28. 71. 35. 68. 29. 26. 20.  0. 29.  4. 40. 51. 34.  8. 27. 72. 91. 40.
  27. 83.]
 [17. 25. 19. 15. 98. 85. 59. 29.  0. 63. 50. 82. 58. 18. 33. 17. 31. 95.
  71. 68.]
 [94. 91. 27. 48. 37. 34. 48.  4. 63.  0. 33. 95. 74. 54. 74. 51. 46. 28.
  17. 65.]
 [13. 83. 97. 10. 10. 89. 34. 40. 50. 33.  0. 63. 11. 96.  6. 14. 19. 80.
  20. 87.]
 [86. 89. 43. 70. 29. 87. 81. 51. 82. 95. 63.  0. 54. 76.  8. 49.

# Brute force

In [147]:
def brute_force(n, k, edges, picked = set()) :
    '''
        input
            n = number of nodes 
            k = number of nodes to pick 
            picked = already picked nodes
        output 
            ans = best answer that contains the nodes of 'picked' 
    ''' 
    n = edges.shape[0] 
    if len(picked) == k :
        ans = 0 
        for u in picked : 
            for v in picked : 
                if v > u : 
                    ans += edges[u][v]
        return ans 
    
    ans = 0
    for v in range(n) : 
        if v not in picked : 
             picked.add(v) 
             ans = max(ans, brute_force(n, k, edges, picked))
             picked.remove(v)
    return ans 

In [148]:
k = 5 
ans = brute_force(n, k, edges)
print(ans)

879.0


# Hill climbing

In [149]:
def random_choice(n, k):
    ''' 
        choose k random unique number between 1 to n
    '''  
    output = random.sample(range(n), k)
    return output

In [150]:
def get_value(state, edges):
    '''
        state is a list that contains some nodes
        return sum of edges of state nodes
    ''' 
    ans = 0
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            ans += edges[state[i]][state[j]]
    
    return ans


We consider two states like X and Y as neighbor states, if there is exactly one node in X that isn't in Y and there is exactly one node in Y that isn't in X. 

For instance [2,3,4,5,6] and [2,3,4,6,7] are neighbors but [2,3,4,5,6] and [1,2,3,7,8] aren't neighbors.

In [151]:
def get_neighbors(n, state):
    '''
        return neighbors of state
    '''
    neighbors = []
    for i in range(len(state)):
        for node in range(n):
            if node not in state:
                new_state = state[:]
                new_state[i] = node
                neighbors.append(new_state)
    return neighbors


In [152]:
def hill_climbing(n, k, edges, num_iters = 10000):
    '''
        input
            n = number of graph nodes 
            k = number of nodes to pick 
            edges = graph edges weights
            num_iters = maximum number of iterations
        output 
            best_value = best state value
    '''
    current_state = random_choice(n,k)
    current_value = get_value(current_state,edges)
    
    while True:
        neighbors = get_neighbors(n, current_state)
        best_neighbor = None
        best_value = current_value
        
        for neighbor in neighbors:
            neighbor_value = get_value(current_state,edges)
            if neighbor_value > best_value:
                best_value = neighbor_value
                best_neighbor = neighbor
        
        if best_neighbor is None:
            break
        
        current_state = best_neighbor
        current_value = best_value
    return current_state

In [153]:
ans = hill_climbing(n, k, edges)
print(ans)

[9, 16, 6, 4, 11]


# Simulated annealing

In [154]:
def simulated_annealing(n, k, edges, alpha = 0.9, initial_temp = 100000, max_iteration = 100000, min_temperature = 0.0001):
    '''
    input                                                                   
        alpha is the temperature decay rate                                        
        max_iteration is the maximum number of iteration (termination condition)   
        min_temperature is the minimum temperature (termination condition)         
    output                                                                  
        best state value                                         
    '''
    temperature = initial_temp
    best_value = float('-INF')

    current_state = random_choice(n, k)
    current_value = get_value(current_state, edges)

    best_state = current_state
    best_value = current_value

    while temperature >= min_temperature and max_iteration > 0:
        max_iteration -= 1

        next_state = random.choice(get_neighbors(n, current_state))
        next_value = get_value(next_state, edges)

        delta_value = next_value - current_value

        if delta_value > 0:
            current_state = next_state
            current_value = next_value
        else:
            acceptance_probability = math.exp(delta_value / temperature)
            if random.random() < acceptance_probability:
                current_state = next_state
                current_value = next_value

        if current_value > best_value:
            best_value = current_value
            best_state = current_state

        temperature *= alpha

    return best_value

In [155]:
ans = simulated_annealing(n, k, edges)
print(ans)

770.0


# How much Hill climbing and Simulated annealing answers are close to actual answers?

In [156]:
k = 5 
seeds = [10, 20, 30, 40, 50, 60, 70, 80, 142, 2024]
brute_force_result = []
hill_climbing_result = []
simulated_annealing_result = []
for seed in seeds: 
    edges = Generator(n, seed)
    brute_force_result.append(brute_force(n, k, edges))
    hill_climbing_result.append(hill_climbing(n, k, edges))
    simulated_annealing_result.append(simulated_annealing(n, k, edges))


In [157]:
print('brute_force_result         :', brute_force_result)
print('hill_climbing_result       :', hill_climbing_result)
print('simulated_annealing_result :', simulated_annealing_result)

brute_force_result         : [785.0, 810.0, 819.0, 783.0, 832.0, 789.0, 813.0, 763.0, 818.0, 860.0]
hill_climbing_result       : [[16, 19, 4, 7, 10], [1, 12, 11, 9, 18], [5, 3, 7, 10, 15], [3, 5, 18, 0, 16], [10, 12, 8, 16, 3], [17, 12, 19, 9, 0], [7, 9, 14, 16, 1], [12, 17, 2, 14, 15], [7, 14, 19, 3, 5], [19, 17, 0, 3, 5]]
simulated_annealing_result : [785.0, 780.0, 777.0, 719.0, 807.0, 769.0, 746.0, 761.0, 818.0, 752.0]
